<a href="https://colab.research.google.com/github/leonardojfrenner/llm_hugging_face_langChain/blob/main/llm_open_source_models_langChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers einops accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.2 MB/s eta 0:00:00


In [2]:
!pip install -q langchain
!pip install -q langchain-community
!pip install -q langchain-huggingface
!pip install -q langchainhub
!pip install -q langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.4 MB/s eta 0:00:0

In [3]:
import torch
import os
import getpass

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [14]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


In [15]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [16]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [17]:
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [18]:
input = "Qual foi a primeira linguagem de programação?"

output = llm.invoke(input)

print(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 A resposta é um pouco complexa, pois existem várias definições de "linguagem de programação" e diferentes perspectivas sobre o que constitui uma linguagem de programação.

No entanto, é comummente aceito que a primeira linguagem de programação seja o Plankalkül, desenvolvido em 1942 pelo matemático alemão Konrad Zuse. O Plankalkül era uma linguagem de programação para calcular tabelas de números e realizar operações matemáticas.

Outras fontes sugerem que a primeira linguagem de programação seja o Short Code, desenvolvido em 1936 por Charles Babbage, um inventor britânico. No entanto, o Short Code não era uma linguagem de programação no sentido moderno, mas sim um sistema de notação para programar máquinas calculadoras.

A linguagem de programação mais antiga que ainda é utilizada hoje em dia é o COBOL (COmmon Business Oriented Language), desenvolvido em 1959. Ele foi projetado para ser fácil de usar e entender, e foi amplamente utilizado em aplicações comerciais durante décadas.

Alé

In [19]:
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

In [20]:
system_prompt = "Você é um assistente e está respondendo perguntas gerais."
user_prompt = input

In [21]:
prompt_template = template.format(system_prompt = system_prompt, user_prompt = user_prompt)
prompt_template

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nQual foi a primeira linguagem de programação?\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [22]:
output = llm.invoke(prompt_template)
output

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Uma pergunta clássica!\n\nA primeira linguagem de programação reconhecida como tal é o Plankalkül, desenvolvido em 1942 pelo matemático alemão Konrad Zuse. No entanto, essa linguagem não era uma linguagem de alto nível, mas sim uma linguagem de máquina específica para o seu computador.\n\nA primeira linguagem de programação de alto nível é considerada ser o Short Code, criado em 1947 por Alan Turing e seus colegas no Laboratório Nacional de Física de Los Alamos (EUA). O Short Code foi projetado para ser mais fácil de usar do que as linguagens de baixo nível da época e permitia ao usuário escrever programas para calcular tabelas de logaritmos e realizar outras tarefas matemáticas.\n\nOutra linguagem importante é o Assembly Language, desenvolvida na década de 1950, que foi usada para programar os primeiros computadores comerciais.\n\nJá a primeira linguagem de programação high-level, com sintaxe similar à linguagem moderna, é considerada ser o COBOL (1959), desenvolvido pela equipe de D